In [1]:
import torch
import torchvision
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plta
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import torchvision.transforms.functional as fn
from torch import nn
import torch.nn.functional as F
import wandb
import torch.optim as optim
from tqdm import tqdm
import sys
# import autoreload
%load_ext autoreload
%autoreload 2
sys.path.append('../scripts/')
from save_model_every_epoch_torch import save_model
import warnings
warnings.filterwarnings("ignore")
from metrics import find_metrics, find_metrics_macro
from save_model_every_epoch_torch import save_model
import matplotlib.pyplot as plt

In [2]:
multiple_gpus = True
if torch.cuda.is_available():
    if torch.cuda.device_count() > 1:
        multiple_gpus = True
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
csv_file = pd.read_csv('../../files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../files/train_images/' + x)

In [4]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size, random_state=42)
    train, val = train_test_split(train, test_size=test_size, random_state=42)
    return train, val, test

In [5]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [6]:
encoder = encoding_data(csv_file)

In [7]:
train, val, test = split_datasets(csv_file, test_size = 0.01)

In [8]:
def get_keys_to_augment(encoder, csv_file, exclude = 5):
    encoded = encoder.transform(csv_file['species'].values.reshape(-1, 1)).toarray()
    mapper = dict(zip(np.unique(np.argmax(encoded, axis = 1), return_counts = True)[0], np.unique(np.argmax(encoded, axis = 1), return_counts = True)[1]))
    total = np.sum(np.array(list(mapper.values())))
    percents = {}
    for k, vals in mapper.items():
        percents[k] = vals / total
    sorted_ones = {k: v for k, v in sorted(percents.items(), key=lambda item: item[1])}
    temp = list(sorted_ones.keys())
    temp = set(temp[:exclude])
    return temp

In [9]:
class DolphinDataset(Dataset):
    def __init__(self, csv_file, encoder, image_size, exclude_classes = 5):
        self.csv_file = csv_file
        self.images = csv_file['updated_paths'].values.tolist()
        self.labels = csv_file['species'].values.tolist()
        self.encoder = encoder
        self.encoded_labels = self.encoder.transform(np.array(self.labels).reshape(-1, 1)).toarray()
        self.convert_rgb = torchvision.transforms.Lambda(lambda x: x.repeat(3, 1, 1))
        self.image_size = image_size

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.images[idx])
        img = torchvision.io.decode_jpeg(img)
        if img.shape[0] != 3:
            img = self.convert_rgb(img)
        img = torchvision.transforms.functional.resize(img, (self.image_size[0], self.image_size[0]))
        img = img / 255.0
        return img, torch.Tensor(self.encoded_labels[idx])


class DolphinDatasetTrain(Dataset):
    def __init__(self, csv_file, encoder, image_size, exclude_classes = 5):
        self.csv_file = csv_file
        self.images = csv_file['updated_paths'].values.tolist()
        self.labels = csv_file['species'].values.tolist()
        self.encoder = encoder
        self.encoded_labels = self.encoder.transform(np.array(self.labels).reshape(-1, 1)).toarray()
        self.convert_rgb = torchvision.transforms.Lambda(lambda x: x.repeat(3, 1, 1))
        self.image_size = image_size
        self.exclude_classes = exclude_classes
        self.exclude = get_keys_to_augment(self.encoder, self.csv_file, self.exclude_classes)
        self.reset_every_epoch()

    def reset_every_epoch(self):
        self.exclude = get_keys_to_augment(self.encoder, self.csv_file, self.exclude_classes)
        index_to_exclude = np.argwhere(np.isin(np.argmax(self.encoded_labels, axis = 1), np.array(list(self.exclude)))).ravel()
        new_values = np.random.randint(0, 2, size=(self.__len__()))
        new_values[index_to_exclude] = -1
        self.new_values = new_values

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.images[idx])
        img = torchvision.io.decode_jpeg(img)
        if img.shape[0] != 3:
            img = self.convert_rgb(img)
        if self.new_values[idx] != -1:
            if self.new_values[idx] == 1:
                img = torchvision.transforms.functional.hflip(img)
        img = torchvision.transforms.functional.resize(img, (self.image_size[0], self.image_size[0]))
        img = img / 255.0
        return img, torch.Tensor(self.encoded_labels[idx])

In [10]:
def get_model():
    incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)
    incep_model.fc = nn.Linear(2048, 30)
    return incep_model

In [11]:
categorical_cross_entropy = nn.CrossEntropyLoss()

Without momentum changes

In [12]:
def train_model(train_dataset, val_dataset, epochs, batch_size, weights_path, load_weights=None):
    model = get_model()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.5)
    wandb.init(project='dolphin',
               config = {
                   'arch' : 'InceptionV4',
                   'batch_size' : batch_size,
                   'epochs' : epochs,
                   'optimizer' : type(optimizer).__name__,
                   'learning rate' : optimizer.param_groups[0]['lr'],
                   'momentum' : optimizer.param_groups[0]['momentum'],
                   'loss_function' : categorical_cross_entropy,
                   'classes' : model.fc.out_features,
                   'Data aug' : 'horizontal_flip'
               })
    data_pointers = {
        'train' : train_dataset,
        'val' : val_dataset,
    }
    if load_weights != None:
        model.load_state_dict(torch.load(load_weights)['model_state_dict'])
    if next(model.parameters()).is_cuda == False:
        if multiple_gpus == True:
            model = nn.DataParallel(model)
        model = model.to(device)
    for epoch in range(epochs):
        train_loss= 0.0
        train_prec, train_rec = 0.0, 0.0
        val_loss = 0.0
        val_prec, val_rec = 0.0, 0.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss, running_precision, running_recall = 0.0, 0.0, 0.0
            with tqdm(data_pointers[phase], unit='batch') as tepoch:
                for imgs, labels in tepoch:
                    tepoch.set_description(f'Epoch: {epoch}')
                    imgs = imgs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            outputs = model(imgs)[0]
                        else:
                            outputs = model(imgs)
                        loss = categorical_cross_entropy(outputs, labels)
                        prec, rec = find_metrics_macro(outputs, labels, batch_size=batch_size)
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                    running_loss += loss.item()
                    running_precision += prec.item()
                    running_recall += rec.item()
                    tepoch.set_postfix(loss = loss.item(), precision = prec.item(), recall = rec.item())
            if phase == 'train':
                train_loss = running_loss / len(train_dataset)
                train_prec = running_precision / len(train_dataset)
                train_rec = running_recall / len(train_dataset)
                print(f'{phase} Loss: {float(train_loss)}')
                print(f'{phase} Precision: {float(train_prec)}')
                print(f'{phase} Recall: {float(train_rec)}')
            else:
                val_loss = running_loss / len(val_dataset)
                val_prec = running_precision / len(val_dataset)
                val_rec = running_recall / len(val_dataset)
                print(f'{phase} Loss: {float(val_loss)}')
                print(f'{phase} Precision: {float(val_prec)}')
                print(f'{phase} Recall: {float(val_rec)}')
        wandb.log({
            'train_loss' : train_loss,
            'val_loss' : val_loss,
            'train_precision' : train_prec,
            'train_recall' : train_rec,
            'val_precision' : val_prec,
            'val_recall' : val_rec
        })
        train_dataset.dataset.reset_every_epoch()
        save_model(model, epoch, optimizer, multiple_gpus, weights_path)

In [14]:
batch_size = 356
train_dataset = DolphinDatasetTrain(train, encoder, image_size = (512, 512))
val_dataset = DolphinDataset(val, encoder, image_size = (512, 512))
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 22, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 22, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum_aug/', epochs=50)

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch: 0: 100%|██████████| 141/141 [03:56<00:00,  1.67s/batch, loss=2.58, precision=0.039, recall=0.0224] 


train Loss: 2.8336530898479704
train Precision: 0.03774534043003904
train Recall: 0.026219382736805484


Epoch: 0: 100%|██████████| 2/2 [00:26<00:00, 13.28s/batch, loss=2.68, precision=0.0404, recall=0.0346]


val Loss: 2.6824615001678467
val Precision: 0.03798512555658817
val Recall: 0.024767922703176737
Weight saved for epoch 0.


Epoch: 1:  40%|███▉      | 56/141 [01:56<02:07,  1.50s/batch, loss=2.52, precision=0.0383, recall=0.0209] 

In [13]:
batch_size = 356
train_dataset = DolphinDatasetTrain(train, encoder, image_size = (512, 512))
val_dataset = DolphinDataset(val, encoder, image_size = (512, 512))
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 22, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 22, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum_aug_v2/', epochs=50, load_weights='../../weights/torch_with_momentum_aug/49.pth')

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch: 0: 100%|██████████| 141/141 [03:50<00:00,  1.63s/batch, loss=1.29, precision=0.22, recall=0.208] 


train Loss: 1.261938899966842
train Precision: 0.2403520513722237
train Recall: 0.23635431278681923


Epoch: 0: 100%|██████████| 2/2 [00:24<00:00, 12.12s/batch, loss=1.12, precision=0.263, recall=0.248]


val Loss: 1.20071941614151
val Precision: 0.26224854588508606
val Recall: 0.2390640750527382
Weight saved for epoch 0.


...

val Loss: 0.9057460427284241
val Precision: 0.32836954295635223
val Recall: 0.33723416924476624
Weight saved for epoch 48.


Epoch: 49: 100%|██████████| 141/141 [03:45<00:00,  1.60s/batch, loss=0.93, precision=0.34, recall=0.387]  


train Loss: 0.9323275025854719
train Precision: 0.31385897469858753
train Recall: 0.3461677689924308


Epoch: 49: 100%|██████████| 2/2 [00:24<00:00, 12.34s/batch, loss=0.901, precision=0.305, recall=0.293]


val Loss: 0.9707867205142975
val Precision: 0.30534350872039795
val Recall: 0.28557781875133514
Weight saved for epoch 49.


In [14]:
batch_size = 356
train_dataset = DolphinDatasetTrain(train, encoder, image_size = (512, 512))
val_dataset = DolphinDataset(val, encoder, image_size = (512, 512))
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 22, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 22, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum_aug_v3/', epochs=50, load_weights='../../weights/torch_with_momentum_aug_v2/49.pth')

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0


train_loss,███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
train_precision,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_recall,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▄▅▅▅▅▅▆▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▃▃▅▃▃▃▃▃▃▃▅▃▃▃▂▂▂▂█▃▃▂▂▂▂▁▂▄▂▂▂▁▂▃▁▁▂▂▂▁
val_precision,▄▄▃▄▄▄▄▄▂▄▃▅▂▅▅▅▄▆▁▅▄▆▄▆▆▆▅▃▆▅▇▇▇▅▇▇▆▄█▇
val_recall,▄▃▁▂▄▃▄▃▃▄▄▄▃▅▄▄▅▄▁▄▃▇▆▅▆▅▆▁▆▆▆▆▇▅▇█▇▆▇▆
train_loss,0.93233
train_precision,0.31386
train_recall,0.34617
val_loss,0.97079
val_precision,0.30534


Epoch: 0: 100%|██████████| 141/141 [03:47<00:00,  1.61s/batch, loss=0.943, precision=0.297, recall=0.311]


train Loss: 0.9273325111849088
train Precision: 0.31588173377598433
train Recall: 0.3461521861823738


Epoch: 0: 100%|██████████| 2/2 [00:24<00:00, 12.46s/batch, loss=0.815, precision=0.332, recall=0.368]


...

val Loss: 0.8933310210704803
val Precision: 0.35294216871261597
val Recall: 0.37517449259757996
Weight saved for epoch 48.


Epoch: 49: 100%|██████████| 141/141 [03:44<00:00,  1.59s/batch, loss=0.856, precision=0.322, recall=0.319]


train Loss: 0.7334827083222409
train Precision: 0.3806634672144626
train Recall: 0.4095574773795216


Epoch: 49: 100%|██████████| 2/2 [00:25<00:00, 12.82s/batch, loss=0.578, precision=0.481, recall=0.466]


val Loss: 0.681388795375824
val Precision: 0.43639303743839264
val Recall: 0.4552062749862671
Weight saved for epoch 49.


In [15]:
batch_size = 356
train_dataset = DolphinDatasetTrain(train, encoder, image_size = (512, 512))
val_dataset = DolphinDataset(val, encoder, image_size = (512, 512))
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 22, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 22, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum_aug_v4/', epochs=25, load_weights='../../weights/torch_with_momentum_aug_v3/49.pth')

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0


train_loss,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
train_precision,▁▁▁▂▂▂▂▂▃▃▃▃▄▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
train_recall,▁▂▁▂▁▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇█▇██
val_loss,▄▅▄▄▄▄▄▆█▃▄▃▅▅▆▅▅▅▃▅▃▂▄▃▃▃▃▃▃▃▃▃▄▇▂▅▅▂▂▁
val_precision,▂▂▃▄▃▄▄▂▁▄▃▄▄▄▄▂▁▃▅▄▅▄▄▅▄▄▆▅▄▄▃▅▆▅▅▅▄▅▆█
val_recall,▄▃▃▅▄▄▄▃▁▅▄▆▅▅▄▄▅▄▆▄▆▆▅▇▅▆▇▆▆▆▅▆▆▃▅▄▅▆▇█
train_loss,0.73348
train_precision,0.38066
train_recall,0.40956
val_loss,0.68139
val_precision,0.43639


Epoch: 0: 100%|██████████| 141/141 [03:43<00:00,  1.59s/batch, loss=0.774, precision=0.418, recall=0.42] 


train Loss: 0.7288095046442451
train Precision: 0.38659233975072277
train Recall: 0.4133008065375876


Epoch: 0: 100%|██████████| 2/2 [00:24<00:00, 12.46s/batch, loss=0.767, precision=0.446, recall=0.413]


...

val Loss: 0.7084128260612488
val Precision: 0.3929676413536072
val Recall: 0.4197677671909332
Weight saved for epoch 23.


Epoch: 24: 100%|██████████| 141/141 [03:50<00:00,  1.63s/batch, loss=0.825, precision=0.398, recall=0.424]


train Loss: 0.6554016234181451
train Precision: 0.4148135322628292
train Recall: 0.44165174272043484


Epoch: 24: 100%|██████████| 2/2 [00:24<00:00, 12.46s/batch, loss=0.67, precision=0.444, recall=0.431] 


val Loss: 0.7559780478477478
val Precision: 0.41286805272102356
val Recall: 0.4178132712841034
Weight saved for epoch 24.
